### Load Required libraries

In [1]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras import backend as K

import numpy as np
import pandas as pd
import os
import PIL
from PIL import Image

Using TensorFlow backend.


### To show Multiline Output within single window

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

###  Create a structure for Convolutional Neural Network

In [3]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D(padding=(1,1),input_shape=(224,224,3)))
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Convolution2D(filters=128, kernel_size=(3, 3), activation='relu'))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Convolution2D(filters=128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Convolution2D(filters=256, kernel_size=(3, 3), activation='relu'))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Convolution2D(filters=256, kernel_size=(3, 3), activation='relu'))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Convolution2D(filters=256, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Convolution2D(filters=512, kernel_size=(3, 3), activation='relu'))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Convolution2D(filters=512, kernel_size=(3, 3), activation='relu'))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Convolution2D(filters=512, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Convolution2D(filters=512, kernel_size=(3, 3), activation='relu'))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Convolution2D(filters=512, kernel_size=(3, 3), activation='relu'))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Convolution2D(filters=512, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(units=4096, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(units=4096, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(units=1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

### Create VGG network

In [4]:
model = VGG_16('vgg16_weights_tf_dim_ordering_tf_kernels.h5')
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

### Looking at the summary of the model

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
__________

### Checking the layers and length of the layers length

In [6]:
model.layers
len(model.layers)

37

### Create a function to read a single image

In [7]:
size = 224
def read_image(f):
    im = Image.open(f)
    im = im.resize((size, size), PIL.Image.NEAREST)
    im = np.asarray(im, dtype='float64')
    im[:,:,0] -= 103.939  ## Mean value for individual channel (RGB) 
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    im = im.reshape(1,224,224,3)
    
    return(im)

### Read a single image from the file

In [8]:
im = read_image('1.jpg')
print(im.shape)

(1, 224, 224, 3)


In [9]:
# with a Sequential model
get_33rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[32].output])

In [10]:
layer_output = get_33rd_layer_output([im])[0]
print(layer_output.shape)

(1, 4096)


###  Preparing the train dataset

In [13]:
img_files = []
for root, dirs, files in os.walk('C:/Users/NY 5211/Downloads/DeepLearningAssignment/train/'):
    img_files.extend(files)
    #print(files)
    #len(files)
    #for name in dirs:
        #print(os.path.join(root,name))

### Check the length of the image file 

In [14]:
#len(img_files)
#print(img_files)#
#print(root)
#print(files)

### Reading all images from the file

In [15]:
n_files = len(img_files)
print('Total number of images:', n_files)

train_size = 120
sample_size = min(n_files,train_size)
print('Sample images taken:', sample_size)
sample = [img_files[i] for i in np.random.choice(len(files),size=sample_size, replace=True)]
i=0
print("Sample Length",len(sample))
x_train = []
y_train = []
print('Reading train images ...')
for file in sample:
    if(i%24==0):
        print('Read {} images'.format(i))
    im = read_image('C:/Users/NY 5211/Downloads/DeepLearningAssignment/train/club_train/'+file)
    temp = get_33rd_layer_output([im])[0]
    x_train.append(temp[0])
    if file[:2]=='c0':
        y_train.extend(['0'])
    elif file[:2]=='c1':
        y_train.extend(['1'])
    elif file[:2]=='c2':
        y_train.extend(['2'])             
    elif file[:2]=='c3':
        y_train.extend(['3'])
    elif file[:2]=='c4':
        y_train.extend(['4'])
    i+=1

print("Done reading all images")



Total number of images: 120
Sample images taken: 120
Sample Length 120
Reading train images ...
Read 0 images
Read 24 images
Read 48 images
Read 72 images
Read 96 images
Done reading all images


###  Check the shape of the both file

In [16]:
x_train = np.array(x_train)
print(x_train.shape, 'x_train Shape')

y_train = to_categorical(y_train)
print(y_train.shape, 'y_train Shape')
y_train

(120, 4096) x_train Shape
(120, 5) y_train Shape


array([[ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  

###  Build a normal MLP model for given data

In [17]:
mlp = Sequential()
mlp.add(Dense(units=1024, input_dim=4096, activation='relu'))
mlp.add(Dropout(rate=0.1))
mlp.add(Dense(units=512, activation='relu'))
mlp.add(Dropout(rate=0.1))
mlp.add(Dense(units=5, activation='softmax'))

from keras.optimizers import Adam
adam = Adam(lr=0.01)
mlp.compile(optimizer=adam, 
            loss='categorical_crossentropy', 
            metrics=['acc'])

In [18]:
mlp.fit(x_train, y_train, epochs=15)

Epoch 1/15
120/120 [==============================] - 3s 25ms/step - loss: 10.5895 - acc: 0.2250
Epoch 2/15
120/120 [==============================] - 1s 9ms/step - loss: 12.4915 - acc: 0.2250
Epoch 3/15
120/120 [==============================] - 1s 8ms/step - loss: 12.6258 - acc: 0.2167
Epoch 4/15
120/120 [==============================] - 1s 7ms/step - loss: 12.6258 - acc: 0.2167
Epoch 5/15
120/120 [==============================] - 1s 6ms/step - loss: 12.6258 - acc: 0.2167
Epoch 6/15
120/120 [==============================] - 1s 6ms/step - loss: 12.6258 - acc: 0.2167
Epoch 7/15
120/120 [==============================] - 1s 5ms/step - loss: 12.6258 - acc: 0.2167
Epoch 8/15
120/120 [==============================] - 1s 5ms/step - loss: 12.6258 - acc: 0.2167
Epoch 9/15
120/120 [==============================] - 1s 5ms/step - loss: 12.6258 - acc: 0.2167
Epoch 10/15
120/120 [==============================] - 1s 5ms/step - loss: 12.6258 - acc: 0.2167
Epoch 11/15
120/120 [=================

###  Preparing the test data

In [19]:
img_test_file = []
for test_root, test_dirs, test_files in os.walk('C:/Users/NY 5211/Downloads/DeepLearningAssignment/test/'):
    img_test_file.extend(test_files)

In [20]:
print(img_test_file)
print(len(test_files))
print(test_files)

['img_100026.jpg', 'img_100050.jpg', 'img_100074.jpg', 'img_100145.jpg', 'img_100191.jpg', 'img_100257.jpg', 'img_100312.jpg', 'img_91724.jpg', 'img_91729.jpg', 'img_91761.jpg', 'img_91769.jpg', 'img_91804.jpg', 'img_91849.jpg', 'img_91906.jpg', 'img_91933.jpg', 'img_91939.jpg', 'img_91993.jpg', 'img_92009.jpg', 'img_92022.jpg', 'img_92088.jpg', 'img_93664.jpg', 'img_93682.jpg', 'img_93732.jpg', 'img_93755.jpg', 'img_93846.jpg', 'img_93875.jpg', 'img_93881.jpg', 'img_93900.jpg', 'img_93970.jpg', 'img_93987.jpg', 'img_94040.jpg', 'img_94092.jpg', 'img_94136.jpg', 'img_96710.jpg', 'img_96719.jpg', 'img_96770.jpg', 'img_96824.jpg', 'img_96924.jpg', 'img_97064.jpg', 'img_97132.jpg', 'img_97201.jpg', 'img_97226.jpg', 'img_97243.jpg', 'img_97326.jpg', 'img_97380.jpg', 'img_97440.jpg', 'img_98387.jpg', 'img_98440.jpg', 'img_98489.jpg', 'img_98507.jpg', 'img_98532.jpg', 'img_98564.jpg', 'img_98634.jpg', 'img_98647.jpg', 'img_98739.jpg', 'img_98746.jpg', 'img_98787.jpg', 'img_98916.jpg', 'img_9

###  Reading the all images from the test file

In [21]:
n_test_files = len(img_test_file)
print('Total number of images:', n_test_files)

test_size = 65
sample_test_size = min(n_test_files,test_size)
print('Sample images taken:', sample_test_size)
sample_test = [img_test_file[i] for i in np.random.choice(len(test_files),size=sample_test_size, replace=True)]
i=0
print("Sample Length",len(sample_test))
x_test = []
print('Reading train images ...')
for file_test in sample_test:
    im1 = read_image('C:/Users/NY 5211/Downloads/DeepLearningAssignment/test/'+file_test)
    temp1 = get_33rd_layer_output([im1])[0]
    x_test.append(temp1[0])

Total number of images: 65
Sample images taken: 65
Sample Length 65
Reading train images ...


### Predict the data using predict_classes

In [112]:
pred = mlp.predict_classes(np.array(x_test))
pred

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

###   Convert Dependent variable as per the given format

In [127]:
ans = np.array(pred).tolist() ## Convert Numpy array list to simple list
newlist = []

for i in ans:
    if i == 2:
        i = 'c2'
    elif i == 3:
        i = 'c3'
    elif i == 4:
        i = 'c4'
    elif i == 0:
        i = 'c0'
    elif i == 1:
        i = 'c1'
    newlist.append(i)
    
ans = newlist
len(ans)       
#print(ans)

65

### Convert the list into data frame

In [128]:
sub_frame = pd.DataFrame({'FileName': test_files, 'Label':ans})
len(sub_frame)

65

###  Generates the final submission csv

In [129]:
submission = pd.DataFrame.to_csv(sub_frame,"Submission.csv", sep=",", index=False)